# Exercício
No diretório `aula02/datasets` disponibilizamos o arquivo `weather_history.csv`. Esse arquivo descreve eventos naturais, como: Temperatura, umidade, velocidade do vento, etc. Existe uma relação entre umidade e temperatura? E quanto a umidade e temperatura aparente? Você pode prever a temperatura aparente dada a umidade?


In [62]:
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession, SQLContext, Row

#Normalização dos dados
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#Regressão
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

from pyspark.ml.feature import StandardScaler

## Leitura dos Dados

In [63]:
spark = SparkSession.builder.appName("dataset").getOrCreate()
dataset = spark.read.csv("weather_history.csv", header=True).cache()

dataset.printSchema()

root
 |-- Formatted Date: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Precip Type: string (nullable = true)
 |-- TempC: string (nullable = true)
 |-- ApTempC: string (nullable = true)
 |-- Humidity: string (nullable = true)
 |-- Wind Speed (km/h): string (nullable = true)
 |-- Wind Bearing (degrees): string (nullable = true)
 |-- Visibility (km): string (nullable = true)
 |-- Loud Cover: string (nullable = true)
 |-- Pressure (millibars): string (nullable = true)
 |-- Daily Summary: string (nullable = true)



In [64]:
data_tarefa = dataset.select(dataset.TempC.cast('float'), dataset.Humidity.cast('float'))

data_tarefa.show(5)

+--------+--------+
|   TempC|Humidity|
+--------+--------+
|9.472222|    0.89|
|9.355556|    0.86|
|9.377778|    0.89|
|8.288889|    0.83|
|8.755555|    0.83|
+--------+--------+
only showing top 5 rows



## Avaliando Correlação Humidade vs Temperatura

In [65]:
#"vetor spark", permite a associação dos dados em um vetor trabalhavel pelo spark
assembler = VectorAssembler(inputCols=["TempC", "Humidity"], outputCol="features")

#determina a sequência de execução
pipeline = Pipeline(stages=[assembler])
# .fit: em cima dos dados de treino; trabalho com tudo
model = pipeline.fit(data_tarefa)
# .transform: em cima dos dados de teste
features = model.transform(data_tarefa)

features.show(5, truncate=False)

+--------+--------+--------------------------------------+
|TempC   |Humidity|features                              |
+--------+--------+--------------------------------------+
|9.472222|0.89    |[9.472222328186035,0.8899999856948853]|
|9.355556|0.86    |[9.355555534362793,0.8600000143051147]|
|9.377778|0.89    |[9.377778053283691,0.8899999856948853]|
|8.288889|0.83    |[8.288888931274414,0.8299999833106995]|
|8.755555|0.83    |[8.755555152893066,0.8299999833106995]|
+--------+--------+--------------------------------------+
only showing top 5 rows



In [66]:
r = Correlation.corr(features, "features").head()
print("Pearson correlation matrix:\n" + str(r[0]))

Pearson correlation matrix:
DenseMatrix([[ 1.        , -0.63225468],
             [-0.63225468,  1.        ]])


## Avaliando Correlação Temperatura Aparente x Umidade

In [67]:
data_tarefa = dataset.select(dataset.ApTempC.cast('float'), dataset.Humidity.cast('float'))

data_tarefa.show(5)

+---------+--------+
|  ApTempC|Humidity|
+---------+--------+
| 7.388889|    0.89|
| 7.227778|    0.86|
| 9.377778|    0.89|
|5.9444447|    0.83|
| 6.977778|    0.83|
+---------+--------+
only showing top 5 rows



In [68]:
#"vetor spark", permite a associação dos dados em um vetor trabalhavel pelo spark
assembler = VectorAssembler(inputCols=["ApTempC", "Humidity"], outputCol="features")

#determina a sequência de execução
pipeline = Pipeline(stages=[assembler])
# .fit: em cima dos dados de treino; trabalho com tudo
model = pipeline.fit(data_tarefa)
# .transform: em cima dos dados de teste
features = model.transform(data_tarefa)

features.show(5, truncate=False)

+---------+--------+--------------------------------------+
|ApTempC  |Humidity|features                              |
+---------+--------+--------------------------------------+
|7.388889 |0.89    |[7.388888835906982,0.8899999856948853]|
|7.227778 |0.86    |[7.22777795791626,0.8600000143051147] |
|9.377778 |0.89    |[9.377778053283691,0.8899999856948853]|
|5.9444447|0.83    |[5.94444465637207,0.8299999833106995] |
|6.977778 |0.83    |[6.97777795791626,0.8299999833106995] |
+---------+--------+--------------------------------------+
only showing top 5 rows



In [69]:
r = Correlation.corr(features, "features").head()
print("Pearson correlation matrix:\n" + str(r[0]))

Pearson correlation matrix:
DenseMatrix([[ 1.      , -0.602571],
             [-0.602571,  1.      ]])


         Portanto, podemos concluir que a relação Temperatura (aparente) X Umidade é inversa - ou seja, quanto maior a temperatura (aparente), menor a humidade - e não linear. Essa relação não é muito forte (como pode-se observar pela distância do índice aos extremos do intervalo [-1,1], portanto a melhor forma de gerar uma função que se aproxime de uma função dessa relação (se ela existir) será através do método GRB. Além disso, a relação entre Temperatura e Umidade é mais linearmente forte do que a relação Temperatura Aparente X Umidade.

## Aplicando Procedimento adequado

In [70]:
assembler = VectorAssembler(inputCols=["ApTempC", "Humidity"], outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="features", withStd=True, withMean=False)
normalizer = Normalizer(inputCol="features", outputCol="normalized", p=1.0)

pipeline = Pipeline(stages=[assembler, normalizer])
model = pipeline.fit(data_tarefa)
features = model.transform(data_tarefa)

def extract(row):
    return tuple(row.normalized.toArray().tolist())

data = features.rdd.map(extract).toDF(["ApTempC","Humidity"])
data = data.withColumnRenamed('Humidity', 'label')
data = data.withColumn('bias', lit(1))
data.show(5, truncate=False)

+------------------+-------------------+----+
|ApTempC           |label              |bias|
+------------------+-------------------+----+
|0.8924976521761431|0.10750234782385697|1   |
|0.893666713248199 |0.10633328675180097|1   |
|0.9133210727465803|0.08667892725341977|1   |
|0.8774807342215485|0.12251926577845151|1   |
|0.893695749346546 |0.10630425065345397|1   |
+------------------+-------------------+----+
only showing top 5 rows



In [71]:
(train, test) = data.randomSplit([0.7, 0.3])

In [72]:
assemblers = VectorAssembler(inputCols=["ApTempC", "bias"], outputCol="features")

#até aqui igual os outros modelos
regression = GBTRegressor(featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[assemblers, regression])

model = pipeline.fit(train)
predictions = model.transform(test)

In [73]:
predictions.show(5, truncate=False)

+-------------------+--------------------+----+-------------------------+-------------------+
|ApTempC            |label               |bias|features                 |prediction         |
+-------------------+--------------------+----+-------------------------+-------------------+
|-1.0               |0.0                 |1   |[-1.0,1.0]               |0.06961545689442539|
|-1.0               |0.0                 |1   |[-1.0,1.0]               |0.06961545689442539|
|-1.0               |0.0                 |1   |[-1.0,1.0]               |0.06961545689442539|
|-0.9784345060853077|0.02156549391469233 |1   |[-0.9784345060853077,1.0]|0.06961545689442539|
|-0.9755856958329854|0.024414304167014626|1   |[-0.9755856958329854,1.0]|0.06961545689442539|
+-------------------+--------------------+----+-------------------------+-------------------+
only showing top 5 rows



In [74]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.0362502
